In [1]:
import gymnasium as gym
import numpy as np
import time
from stable_baselines3 import A2C
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecFrameStack, VecTransposeImage, DummyVecEnv
import torch.nn as nn

2024-01-30 12:34:45.101114: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 12:34:45.101239: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 12:34:45.137072: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-30 12:34:45.209497: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-30 12:34:46.529825: W tensorflow/compiler/tf2

In [2]:


class CustomObservationWrapper(gym.ObservationWrapper):
    def __init__(self, env, feature_extractor):
        super(CustomObservationWrapper, self).__init__(env)
        # Define the new observation space here, e.g., an image
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=(400, 600, 3), dtype=np.uint8)
        self.observation_space = feature_extractor.output_dim
        self.feature_extractor = feature_extractor

    def observation(self, observation):
        # Process the observation and return it
        # For example, capturing an image from the environment
        #Use model to get features
        
        return self.feature_extractor.extract_futrea()
        
        return self.env.render()




In [ ]:
env = gym.make('CartPole-v1', render_mode="rgb_array")
env = CustomObservationWrapper(env)

# Wrap it into a DummyVecEnv for compatibility with SB3
env = DummyVecEnv([lambda: env])
env = VecTransposeImage(env)  # Required for CNN policies in SB3

model = A2C("CnnPolicy", env, verbose=1)
model.learn(total_timesteps=10000, progress_bar=True)

In [4]:
episodes = 1

env = gym.make('CartPole-v1', render_mode="rgb_array")
env = CustomObservationWrapper(env)

# Wrap it into a DummyVecEnv for compatibility with SB3
env = DummyVecEnv([lambda: env])
env = VecTransposeImage(env) 

for episode in range(1, episodes+1):    
    observation = env.reset()
    print(observation.shape)
    finished = False
    accumulated_reward = 0
    
    while not finished:
        action = model.predict(observation)[0]
        observation, reward, terminated, truncated = env.step(action)
        accumulated_reward += reward

        terminated_flag = terminated[0]
        truncated_flag = truncated[0].get('TimeLimit.truncated', False)
        
        finished = terminated_flag or truncated_flag

        if finished:
            env.reset()
            print("Episode {} terminated".format(episode))
            print("Accumulated Reward: {}".format(accumulated_reward))

env.close()

(4,)
(1, 3, 400, 600)


NameError: name 'model' is not defined